# **Concurrency**

- Concurrency is the process in which there are multiple tasks running and completed during overlapping periods of time

- Requires at least two tasks to exist

- A **thread** is a representation of the actual sequence of processor instructions that are actively being executed

- Useful for **I/O Bound Tasks:** when a program spends time waiting for file reads, network requests, database queries, or API calls

- Global Interpreter Lock (GIL): a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecodes at once (only one thread can use the shared resource at once)

In [21]:
# Imports

import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor

### Normal yfinance API call

In [22]:
%%time

stock = yf.Ticker('TSLA')
expirations = list(stock.options)[:10]

expiry_df_ls = [] # List of different expiry call option data frames
for date in expirations:
    expiry_df_ls.append(pd.DataFrame(stock.option_chain(date).calls))

CPU times: user 76.3 ms, sys: 8.99 ms, total: 85.2 ms
Wall time: 1.8 s


In [23]:
def get_options(stock, date):
    option_chain = stock.option_chain(date)
    return pd.DataFrame(option_chain.calls)

### Threaded yfinance API call

In [24]:
%%time

stock = yf.Ticker('NVDA')
expirations = list(stock.options)[:10]

with ThreadPoolExecutor(max_workers=min(len(expirations), 10)) as executor:
    futures = [executor.submit(get_options, stock, date) for date in expirations]
    expiry_df_ls = [future.result() for future in futures]

CPU times: user 195 ms, sys: 40.4 ms, total: 235 ms
Wall time: 738 ms
